# Lab 1 for EC3318/MN3101 Corporate Finance

**Contents**:

1. Virtual Environments and Jupyter Notebooks
2. Loading stock price data
3. Computing stock returns
4. Visualising stock prices and returns

**Instructions**:

1. Read the notebook
2. Execute code cells as you encounter them
3. Use GitHub Copilot to ask questions on concepts or Python syntax


## 1. Virtual environments and Jupyter Notebooks

**What are virtual environments?**

Virtual environments are like separate workspaces for different projects.

Just as you organize your coursework into folders for each class, virtual environments keep your Python packages organized and separate for different analysis projects.

**Why this matters for working with Python**

Installing packages for one project can break another project's code. Virtual environments ensure each project has exactly the packages it needs.

Key benefits:

- Your code produces consistent results every time
- Projects work seamlessly across different computers (e.g., GitHub Codespaces or your own computer)
- Your code won't break when you install new packages for other courses
- You can reproduce your analysis months later with identical results

**Your current setup**

Your workspace uses a managed virtual environment called **"cflabs"** with all required packages pre-installed. Here's what you see in your files:

- `.python-version` - Specifies your Python version
- `pyproject.toml` - Lists project dependencies
- `uv.lock` - Records exact package versions for reproducibility
- `.ipynb` files in the `lab1/` folder - Your Jupyter Notebooks
- `.venv` actual virtual environment files

![EnvironmentFiles](.pics/pic1_virtual_env_files.png)

This environment is managed with the Environment Manager called [uv](https://docs.astral.sh/uv/).

**Working with Jupyter Notebooks**

Jupyter Notebooks combine code, visualizations, and text in one document. Each `.ipynb` file lets you:

- Execute code in small chunks (cells)
- See results immediately
- Add explanations and analysis
- Create inline charts and graphs

**Selecting the right Python Interpreter**

**If you encounter import errors**, check your interpreter:

1. **In Notebooks**: Look at the top-right corner of the Notebook for the "Select Interpreter" button or Python version
2. **To change**: Click the button or version number → Select "Python 3.x (cflabs)

![EnvironmentPicker](.pics/pic2_virtual_env_picker.png)

Quick fix in Visual Studio Code: Press `Ctrl+Shift+P` → Type "Python: Select Interpreter" → Choose the one with "cflabs"

**Verification checklist**

- Correct interpreter selected: Bottom-right shows "cflabs"
- Notebook kernel: Top-right shows "Python 3.x (cflabs)"
- Can import the essential packages (execute next cell)


In [ ]:
# Import essential data manipulation package
import pandas as pd
import numpy as np

**Ready to start**

Your environment is pre-configured and ready to use. Focus on your analysis - the technical setup is handled for you!


## 2. Loading stock price data

Now we'll work with real stock price data from our Excel file.


In [ ]:
# Load stock price data from Excel file
prices_raw = pd.read_excel(
    io="downloading_stock_prices.xlsx",  # our Excel file
    sheet_name="Data download",  # Sheet in Excel file
    skiprows=7,  # Skip first 7 rows in sheet
)

prices_raw

**Why Excel data matters in finance**

Excel remains ubiquitous in financial practice. Investment banks, asset managers, and corporate finance teams routinely receive data in Excel format from Bloomberg, Refinitiv, and other financial data providers.

Learning to programmatically import Excel data is essential because:

- Manual copy-pasting introduces errors and is not reproducible
- Financial models require regular updates with new data
- Regulatory compliance often demands audit trails of data sources
- Large datasets exceed Excel's row limitations (1,048,576 rows)


In [ ]:
# Import datetime library for working with dates
from datetime import datetime, timedelta

# Convert Excel numeric date to a normal date
prices_raw["date"] = pd.to_datetime(
    prices_raw["date"],  # date column
    unit="D",  # in days
    origin="1899-12-30",  # the date that a number of zero represents
)

prices_raw

In [ ]:
# Shape of data
prices_raw.shape

In [ ]:
# Columns
prices_raw.columns.tolist()

In [ ]:
# Data types
prices_raw.dtypes

In [ ]:
# Date range
[
    prices_raw["date"].min(),  # earliest date in data
    prices_raw["date"].max(),  # latest date
]

In [ ]:
# Check for missing values
prices_raw.isnull().sum()

**Data quality checks: Foundation of financial analysis**

Before analyzing any financial data, rigorous quality checks are mandatory. Missing values, incorrect dates, and outliers can lead to:

- **Incorrect valuations** in DCF models
- **Flawed risk assessments** for portfolio management
- **Regulatory violations** in financial reporting
- **Trading losses** from bad signals

Professional practice requires documenting these checks for audit purposes and regulatory compliance (e.g., Basel III, Solvency II, IFRS 9).


In [ ]:
# Transform from wide format to tidy (long) format

# Melt the dataframe to convert from wide to long format
prices_tidy = pd.melt(
    prices_raw,  # our data
    id_vars=["date"],  # column that identifies rows in wide data
    var_name="symbol",  # new column for current variable names
    value_name="adj_close",  # new column for respective table values
)
prices_tidy

In [ ]:
# Sort by symbol and date for better organization
prices_tidy = (
    prices_tidy.get(["symbol", "date", "adj_close"])  # arrange columns differently
    .sort_values(["symbol", "date"])  # sort rows by symbol and date
    .reset_index(drop=True)  # maintain clean row identifiers
)

prices_tidy

**Tidy data: The foundation for scalable financial analysis**

Converting from "wide" to "long" format may seem technical, but it's crucial for financial modeling:

- **Portfolio analysis**: Compare hundreds of securities simultaneously
- **Risk management**: Calculate correlations across multiple assets
- **Performance attribution**: Analyze returns by sector, geography, or style
- **Regulatory reporting**: Format data for XBRL and other regulatory standards

This structure enables vectorized operations essential for high-frequency analysis and real-time risk monitoring.


## 3. Computing stock returns


**Stock returns: The building block of modern finance**

Returns, not prices, are the fundamental unit of analysis in finance because they are:

- **Scale-free**: A £1 and £100 stock with identical 10% returns have equivalent risk-return profiles
- **Stationary**: Returns exhibit more stable statistical properties than prices
- **Additive**: Multi-period returns can be calculated by compounding single-period returns
- **Comparable**: Essential for portfolio optimization, CAPM, and Fama-French models

The percentage change calculation (`pct_change()`) computes simple returns: $(P_t - P_{t-1})/P_{t-1}$


In [ ]:
# Calculate returns for each stock

returns = (
    prices_tidy.sort_values(["symbol", "date"])  # verify sorted rows
    .assign(
        ret=prices_tidy.groupby("symbol")["adj_close"].pct_change()
    )  # calculate returns
    .reset_index(drop=True)  # clean row identifiers
)

# look at first five rows per stock
returns.groupby("symbol").head()

In [ ]:
# Drop first row for each stock with missing returns
returns = returns.dropna()

# look at first five rows per stock
returns.groupby("symbol").head()

**Handling missing returns: A practical necessity**

The first observation for each stock has no return (can't calculate change from previous period). In practice:

- **IPO data**: New listings have no prior trading history
- **Corporate actions**: Stock splits and dividends create data discontinuities
- **Delisting events**: Companies leaving exchanges create missing data
- **Data vendor issues**: Bloomberg and Reuters occasionally have gaps

Dropping these observations is standard practice, but analysts must document this choice for audit trails.


## 4. Visualising stock prices and returns


**Visualization in financial analysis: Beyond pretty charts**

Effective visualization is critical for:

- **Risk communication** to boards and regulators
- **Pattern recognition** in market anomalies and trading strategies
- **Performance reporting** to institutional investors
- **Stress testing** under different market scenarios

The plots we'll create mirror those used in professional investment management: price charts for performance tracking and return histograms for risk assessment.


In [ ]:
# Focus on Amazon stock
amazon_data = returns.query("symbol == 'AMZN'")  # filter for AMZN symbol
amazon_data

In [ ]:
# Summary statistics
pd.DataFrame(amazon_data["ret"].describe()).round(3).T

In [ ]:
# Yearly summary statistics
(amazon_data["ret"].groupby(amazon_data["date"].dt.year).describe().round(3))

**Summary statistics: The language of risk and return**

These statistics form the foundation of modern portfolio theory:

- **Mean**: Expected return (μ) - central to CAPM and asset pricing
- **Standard deviation**: Volatility (σ) - the classic risk measure
- **Min/Max**: Extreme values indicating tail risk
- **Count**: Sample size affecting statistical significance

Notice how these vary by year - this time-varying risk is why financial models use rolling windows and GARCH specifications.


In [ ]:
# Import plotnine for producing graphs
# See https://plotnine.org
from plotnine import *

In [ ]:
# Plot Amazon stock price
amazon_prices_figure = (
    ggplot(amazon_data, aes(x="date", y="adj_close"))
    + geom_line()
    + labs(x="", y="Adjusted close ($)", title="Amazon (AMZN) stock price")
    + theme(axis_text_x=element_text(rotation=45, hjust=1))
)
amazon_prices_figure

In [ ]:
# package for percentage scales
from mizani.formatters import percent_format

# 95% value-at-risk (VaR)
quantile_05 = amazon_data["ret"].quantile(0.05)

# plot Amazon stock returns histogram with 95% VaR
amazon_returns_figure = (
    ggplot(amazon_data, aes(x="ret"))
    + geom_histogram(bins=100)
    + geom_vline(aes(xintercept=quantile_05), linetype="dashed")
    + labs(x="", y="", title="Distribution of monthly Amazon stock returns")
    + scale_x_continuous(labels=percent_format())
)
amazon_returns_figure

**Value-at-Risk (VaR): Quantifying downside risk**

The vertical dashed line shows the 5% VaR - there's a 5% probability of losing more than this amount in any given month.

VaR is ubiquitous in finance:

- **Regulatory capital**: Basel III requires banks to hold capital based on VaR
- **Risk limits**: Trading desks face daily VaR constraints
- **Portfolio management**: Asset allocations often target specific VaR levels
- **Stress testing**: Central banks use VaR in systemic risk assessment

Note: VaR has limitations (doesn't capture tail risk beyond the threshold) leading to complementary measures like Expected Shortfall.


In [ ]:
# Summary statistics for each stock through groupby-aggregation
summary_stats = (
    returns.groupby("symbol")["ret"]
    .agg(["count", "mean", "std", "min", "max"])
    .round(3)
)
summary_stats

In [ ]:
# Annualized statistics
annualized_stats = (
    returns.groupby("symbol")["ret"]  # group by symbol and focus on returns
    .agg(["mean", "std"])  # get means and standard deviations
    .assign(
        ann_mean=lambda x: x["mean"] * 12,  # annualise mean return
        ann_vol=lambda x: x["std"] * np.sqrt(12),  # annualise volatility
    )
    .get(["ann_mean", "ann_vol"])
    .round(3)
)
annualized_stats

**Annualization: Making returns comparable**

Monthly returns must be annualized for meaningful comparison:

- **Return scaling**: Multiply by 12 (assumes no compounding within year)
- **Volatility scaling**: Multiply by √12 (accounts for volatility's square-root-of-time scaling)

This follows from financial theory:

- Returns scale linearly with time
- Variance scales linearly with time
- Standard deviation (volatility) scales with √time

These annualized figures enable comparison with market benchmarks, peer companies, and required returns from CAPM.


In [ ]:
# more formatting functions for plotnine
from mizani.breaks import date_breaks
from mizani.formatters import date_format

# plot all three stock prices at once
prices_figure = (
    ggplot(returns, aes(y="adj_close", x="date", color="symbol"))
    + geom_line()
    + scale_x_datetime(breaks=date_breaks(width="1 year"), labels=date_format("%Y"))
    + labs(x="", y="", color="", title="Stock prices of AMZN, WMT, KO")
    + theme(legend_position="right")
)
prices_figure.show()

In [ ]:
# Plot returns series for all three stocks
returns_figure = (
    ggplot(returns, aes(y="ret", x="date", color="symbol"))
    + geom_line()
    + scale_x_datetime(breaks=date_breaks(width="1 year"), labels=date_format("%Y"))
    + labs(x="", y="", color="", title="Monthly returns of AMZN, WMT, KO")
    + theme(legend_position="right")
)
returns_figure.show()

**Cross-sectional analysis: The foundation of portfolio theory**

Comparing multiple securities simultaneously reveals:

- **Correlation patterns**: Essential for diversification benefits
- **Relative performance**: Identifying alpha generation opportunities
- **Sector dynamics**: Understanding systematic vs. idiosyncratic risk
- **Mean reversion**: Potential arbitrage opportunities

This multi-asset visualization is how portfolio managers monitor:

- Factor exposures (growth vs. value, large vs. small cap)
- Geographic allocation effects
- Currency hedging decisions
- Rebalancing triggers

The varying correlations visible in these charts drive the benefits of diversification central to modern portfolio theory.
